In [ ]:
pip install praw

In [ ]:
import praw
import time
import datetime
import pandas as pd

str_client_id = 'fYRWm5jkNOpIWA'
str_client_secret = 'zKi7IdnZZ-5xDxw_jc-HBZpEjN0HZQ'
str_user_agent = 'RedBat'
str_file_name = 'Posts.csv'

df_all_posts = pd.read_csv(str_file_name)
# Delete first
if df_all_posts.shape[0] != 0 :
    str_col_name = df_all_posts.columns[0]
    df_all_posts = df_all_posts.drop([str_col_name], axis=1)

dict_posts = {}

if df_all_posts.empty == False:
    for index, post in df_all_posts.iterrows():
        dict_posts[post['Id']] = post

features = ['Subreddit',
            'Id', 
            'Created',
            'Author',
            'Score', 
            'Is_Original_Content', 
            'Num_Comments', 
            'Flair', 
            'Title', 
            'Selftext']

obj_reddit = praw.Reddit(client_id=str_client_id, client_secret=str_client_secret, user_agent=str_user_agent)
int_count = 1

while True:
    obj_subredit = obj_reddit.subreddit('wallstreetbets')
    int_unique = 0
    for post in obj_subredit.new():
        if post.id in dict_posts.keys():
            dict_posts[post.id].Num_Comments = post.num_comments
        else:
            list_posts =[[post.subreddit, 
                          post.id, 
                          datetime.datetime.utcfromtimestamp(post.created),
                          post.author,
                          post.score, 
                          post.is_original_content, 
                          post.num_comments, 
                          post.link_flair_text, 
                          post.title.encode('utf8'), 
                          post.selftext.encode('utf8')]]
                
            df_post = pd.DataFrame(list_posts, columns=features)
            df_all_posts = pd.concat([df_all_posts, df_post])
                
            dict_posts[post.id] = df_post
            int_unique += 1

    df_all_posts.to_csv(str_file_name)
    print('Running! Exectued once at: ' + datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' Total: ' + str(df_all_posts.shape[0]) + ' New: ' + str(int_unique))
    
    int_count += 1
    if(int_count > 9):
        break
    time.sleep(60)
    

In [ ]:
obj_post = obj_reddit.submission(id="l9d89z")

obj_post.comments.replace_more(limit=None)
for comment in obj_post.comments.list():
    print(comment.__dict__)